# Hash the Trash notebook

Here everything should run smoothly with just this notebook, running Ganache (with quickstart) and setting the PORT on which Ganache is running (The contract's abi and bytecode has been compiled from Remix and imported here to run everything).

To see the real time version of the blockchain event log catcher: 
1. Deploy the contract from this notebook [from cell 1 through 6]
2. launch the 'events_real_time.py' from the terminal and just continue through the notebook (basic prints will appear in the terminal when trashbags lifecycle events gets called and the events_log.csv will record them)

Various sections are:
* [deploying the contract](#deploy)
* [operating with its functions from eth accounts](#functions)
    * [TARI](#tari)
    * [Trash cycle](#cycle)
    * [Refund](#refund)
* [picking up logs of certain events from the chain](#logs)

In [ ]:
!pip install -r requirements.txt # if not already run

In [1]:
# imports
import random, time, json
import pandas as pd
from web3 import Web3
from contracts.abi_bytecode import abi, bytecode # saved externally as .py

# to webscrape ETH exchange rate
from bs4 import BeautifulSoup
import requests
from forex_python.converter import CurrencyRates

In [2]:
# Connecting to ganache through opened up PORT
ganache_url = 'HTTP://127.0.0.1:7545'      #change here if different
web3 = Web3(Web3.HTTPProvider(ganache_url))
web3.isConnected()

True

In [3]:
# Compiled abi and bytecode of trash.sol which inherits from citizenz.sol (and Ownable, safemath etc)
abiRemix = json.loads(abi)         # turned to string after copy to clipboard from Remix
bytecodeRemix = bytecode['object'] # it is a dictionary (as copy to clipboard form remix), we use the object for web3 deployment

# simple database example and adding column with ganache accounts
data = pd.read_excel('data/example_data.xlsx', sheet_name='agents_data')
data['address'] = web3.eth.accounts     
data

,name,surname,family,mq,weight,recycle,lat,long,role,address
0,Comune,Codogno,0.0,0.0,0.0,NaN,NaN,NaN,municipality,0xa14Dc72900593770d7aC81478660e91704da60CD
1,Francesca,Bianchessi,2.0,70.0,500.0,NaN,NaN,NaN,citizen,0x20916136FcC20B2cB78FB476156FADB4b1641331
2,Ilaria,Bolla,4.0,200.0,900.0,NaN,NaN,NaN,citizen,0xced646C5b0f38FbF0cca57a174A248907fd1132F
3,Alessandro,Botti,5.0,340.0,800.0,NaN,NaN,NaN,citizen,0x0Dc2998f5b8D55743B5b73756b8d526ce156aA19
4,Davide,Castellini,3.0,50.0,200.0,NaN,NaN,NaN,citizen,0xA91d26a3863004aBd9613E955fF4842f2922A842
5,Anna,Di Marco,4.0,80.0,560.0,NaN,NaN,NaN,citizen,0x132A9dFcBD4192aCE3a9dF87AdBA3FeE2191B7a7
6,Truck,Recycle,NaN,NaN,NaN,True,NaN,NaN,truck,0x87A34C6cB35E8a34B4Dc50f5746B533fB9076F83
7,Truck,Not Recycle,NaN,NaN,NaN,False,NaN,NaN,truck,0x13AF23dce57BBc6Edc73b2B0F71B0925d04A4F9F
8,Disposal Station,Milano,NaN,NaN,NaN,True,45.4642,9.1900,station,0xffE61522eF61ecFDF58020690C2CC340ccE667aC
9,Disposal Station,Bologna,NaN,NaN,NaN,False,44.4949,11.3426,station,0x3E22eb9c92123B1D278D18C371Db54D7b655ed07


<a name = 'deploy'></a>

### Deploying the contract

Here we first create a dictionary with the readily available 10 accounts from ganache and then make the municiaplity deploy the contract on the blockchain

In [4]:
def deploy_contract(deployer, _abi, _bytecode):
    """
    Deploy the contract using Python and web3.py without relying on Remix (aside from getting the compiled abi and bytecode)
    
    Parameters
    -------------
        deployer: eth account
        abi, bytecode : compiled contract things
        
    Returns
    -------------
        contract instance
    """
    contract = web3.eth.contract(abi=_abi, bytecode=_bytecode)      # compiled contract
    tx_hash = contract.constructor().transact({'from': deployer}) # contract constructor call (i.e. deploy)
    
    tx_receipt = web3.eth.waitForTransactionReceipt(tx_hash)      # Get receipt when deployed to blockchain
    print(f"Deployed contract at address:\n{tx_receipt.contractAddress}") # contract address
    
    # simple yet effective method to pass the contract address to the real time filtering
    with open('data/ctr_addr.txt', 'w') as f:
        f.write(tx_receipt.contractAddress)
    
    deployed_ctr = web3.eth.contract(address = tx_receipt.contractAddress, abi  = _abi) # contract
    
    return deployed_ctr

In [5]:
# Deploy contract (municipality in our case) [can check from ganache the blocks to see if it happened]
municipality = data[data.role == 'municipality']['address'].item()

contract = deploy_contract(deployer = municipality, _abi = abiRemix, _bytecode = bytecodeRemix)

Deployed contract at address:
0x2F14C8d0eA57c4b2058f82106C0dbBFcDaD3A9Bc


##### !! Can now run the events_real_time.py !!

<a name = 'functions'></a>
## Interacting with the functions

First a simplle check of the owner of the contract and then creating the Agents.

(Still unsure but i think function(...).**call**(...) works only for 'view' functions; need function(...).**transact**(...) for those that modify the chain)

In [6]:
# simple check for owner of the contract
owner = contract.functions.owner().call() # get owner from contract function
print(f"owner: {owner}")
print(f"Is it the municipality? {owner == data[data.role == 'municipality']['address'].item()}")

# to remember inputs and all functions
contract.all_functions()

owner: 0xa14Dc72900593770d7aC81478660e91704da60CD
Is it the municipality? True


[<Function MunicipalityBalance()>,
 <Function TariAmount(address)>,
 <Function citizens(address)>,
 <Function createCitizen(address,string,uint256,uint256,uint256)>,
 <Function createStation(address,bool,int256,int256)>,
 <Function createTruck(address,bool)>,
 <Function deleteCitizen(address)>,
 <Function deleteStation(address)>,
 <Function deleteTruck(address)>,
 <Function destroyContract()>,
 <Function drop(address,int256,int256)>,
 <Function givePayout(address)>,
 <Function numberC()>,
 <Function numberS()>,
 <Function numberT()>,
 <Function owner()>,
 <Function payTari()>,
 <Function pick(address,uint256,uint256)>,
 <Function received(bool,address,uint256)>,
 <Function renounceOwnership()>,
 <Function setBeginningYear()>,
 <Function showStartTime()>,
 <Function stations(address)>,
 <Function transferOwnership(address)>,
 <Function trucks(address)>,
 <Function withdraw()>]

In [7]:
# Creating list of touples as correct inputs for the contract structs

# Create CITIZENS - (address payable _address, string memory _name, uint _family, uint _house, uint256 _w)
citizensL = [[r.address, " ".join([r.name, r.surname]), int(r.family), int(r.mq), int(r.weight)] 
             for r in data.itertuples() if r.role == 'citizen']

# Create TRUCKS - (address _address, bool _recycle)
trucksL = [[r.address, r.recycle] for r in data.itertuples() if r.role == 'truck']

# Create STATIONS - (address _address, bool _recycle, int _lat, int _long)
stationsL = [[r.address, r.recycle, int(r.lat), int(r.long)] for r in data.itertuples() if r.role == 'station']

print(f'Example of createCitizen inputs: {citizensL[0]}')

Example of createCitizen inputs: ['0x20916136FcC20B2cB78FB476156FADB4b1641331', 'Francesca Bianchessi', 2, 70, 500]


In [8]:
def create(municip_addr ,c, t, s):
    '''
    Calls contract funcions to assigns addresses (and their characteristics) roles and populate mappings
    
    Parameters
    ----------------------
        municip_addr : eth address of the municipality
        c, t, s : list of touples with correct characteristics
        
    Returns
    ----------------------
        nothing on python, the chain grows as these transactions are registered
    '''
    from_dict = {'from': municip_addr}
    for i in range(len(c)):
        contract.functions.createCitizen(c[i][0], c[i][1], c[i][2], c[i][3], c[i][4]).transact(from_dict)
    for j in range(len(t)):
        contract.functions.createTruck(t[j][0], t[j][1]).transact(from_dict)
    for k in range(len(s)):
        contract.functions.createStation(s[k][0], s[k][1], s[k][2], s[k][3]).transact(from_dict)
    print('Done')
    

# Some functions to show structs statuses as we change them
def get_citizen_info(address):
    """
    Call to the contract to pretty print informations on citizen
    """
    info = contract.functions.citizens(address).call()
    print(f"""
    Address eth : {address}
    Name : {info[0]}
    Family members : {info[1]}
    House sq.meters : {info[2]}
    Assigned weight\liters : {info[3]}
    TARI amount : {info[4]}
    Recyclable Waste Tot : {info[5]}
    Non Recyclable Waste Tot : {info[6]}
    Paid TARI : {info[7]}
    Active account : {info[8]}
    """)

def get_truck_info(address):
    """Show pretty info on trucks"""
    info = contract.functions.trucks(address).call()
    print(f"""
    Address eth : {address}
    Truck number : {info[0]}
    Weight transported : {info[1]}
    Recyclable Truck : {info[2]}
    Active Truck : {info[3]}
    """)
    
    
def get_station_info(address):
    """Show pretty info for stations"""
    info = contract.functions.stations(address).call()
    print(f"""
    Address eth : {address}
    Station nr. : {info[0]}
    Weight : {info[1]}
    latitude : {info[2]}
    longitude : {info[3]}
    Recyclable Plant : {info[4]}
    Active Plant : {info[5]}
    """)

In [9]:
# Populating mappings and check amount of entities
create(data[data.role == 'municipality']['address'].item(),citizensL, trucksL, stationsL)

print(f"""
# Citizens: {contract.functions.numberC().call()} 
# Trucks : {contract.functions.numberT().call()}
# PLants : {contract.functions.numberS().call()}""")

Done

# Citizens: 5 
# Trucks : 2
# PLants : 2


In [10]:
# using exemplar citizen and other roles to do checks along the notebook
dutiful_citizen = data[data.role == 'citizen']['address'].reset_index(drop = True)[0]
get_citizen_info(dutiful_citizen)

ex_truck = data[data.role == 'truck']['address'].reset_index(drop = True)[0]
get_truck_info(ex_truck)

ex_station = data[data.role == 'station']['address'].reset_index(drop = True)[0]
get_station_info(ex_station)


    Address eth : 0x20916136FcC20B2cB78FB476156FADB4b1641331
    Name : Francesca Bianchessi
    Family members : 2
    House sq.meters : 70
    Assigned weight\liters : 500
    TARI amount : 0
    Recyclable Waste Tot : 0
    Non Recyclable Waste Tot : 0
    Paid TARI : False
    Active account : True
    

    Address eth : 0x87A34C6cB35E8a34B4Dc50f5746B533fB9076F83
    Truck number : 1
    Weight transported : 0
    Recyclable Truck : True
    Active Truck : True
    

    Address eth : 0xffE61522eF61ecFDF58020690C2CC340ccE667aC
    Station nr. : 1
    Weight : 0
    latitude : 45
    longitude : 9
    Recyclable Plant : True
    Active Plant : True
    


<a name = 'tari'></a>
### Compute TARI 

Takes into account number of household members in the nucleus and square meters of the property

` formula ?`

In [11]:
# Calculate amount of TARI for each citizen (and looping for the others)
for addr in data[data.role == 'citizen']['address']:
    contract.functions.TariAmount(addr).transact({'from' : municipality})

In [12]:
get_citizen_info(dutiful_citizen)
"""Looks expensive"""


    Address eth : 0x20916136FcC20B2cB78FB476156FADB4b1641331
    Name : Francesca Bianchessi
    Family members : 2
    House sq.meters : 70
    Assigned weight\liters : 500
    TARI amount : 190000000000000000
    Recyclable Waste Tot : 0
    Non Recyclable Waste Tot : 0
    Paid TARI : False
    Active account : True
    


'Looks expensive'

In [13]:
# Webscrape currency exchange rates to convert computed TARI in wei to euro
# Save the TARI amount for each citizen 
tari_list = [contract.functions.citizens(addr).call()[4]*(10**(-18)) 
             for addr in data[data.role == 'citizen']['address']]
data.loc[data.role == 'citizen', 'TARI_eth'] = tari_list

# Convert to EUR 
cmc = requests.get('https://coinmarketcap.com/currencies/ethereum/markets/')
soup = BeautifulSoup(cmc.content, 'html.parser')  
data_coinmkt = soup.find('script', type="application/ld+json")
data_coinmkt = json.loads(data_coinmkt.contents[0])
usd_eth = data_coinmkt['currentExchangeRate']['price']

c = CurrencyRates()
usd_eur_rate = c.get_rate('USD', 'EUR') 
eur_eth = usd_eth * usd_eur_rate 

data.loc[data.role == 'citizen', 'TARI_eur'] = data.loc[data.role == 'citizen', 'TARI_eth']*eur_eth 
data[['name', 'surname', 'family', 'mq', 'weight', 'role', 'TARI_eth', 'TARI_eur']]

,name,surname,family,mq,weight,role,TARI_eth,TARI_eur
0,Comune,Codogno,0.0,0.0,0.0,municipality,NaN,NaN
1,Francesca,Bianchessi,2.0,70.0,500.0,citizen,0.190,87.534860
2,Ilaria,Bolla,4.0,200.0,900.0,citizen,0.490,225.747796
3,Alessandro,Botti,5.0,340.0,800.0,citizen,1.440,663.422094
4,Davide,Castellini,3.0,50.0,200.0,citizen,0.120,55.285175
5,Anna,Di Marco,4.0,80.0,560.0,citizen,0.216,99.513314
6,Truck,Recycle,NaN,NaN,NaN,truck,NaN,NaN
7,Truck,Not Recycle,NaN,NaN,NaN,truck,NaN,NaN
8,Disposal Station,Milano,NaN,NaN,NaN,station,NaN,NaN
9,Disposal Station,Bologna,NaN,NaN,NaN,station,NaN,NaN


In [14]:
# Each citizen pays its due amount
for addr in data[data.role == 'citizen']['address']:
    
    info = contract.functions.citizens(addr).call() # need the amount from the contract
    print(f"Citizen {info[0]} is paying {info[4]} wei --> {info[4]*10**(-18)*eur_eth} Euro")
    
    # payable function, the amount is passed in the dictionary of .transact()
    contract.functions.payTari().transact({'from': addr, 'value' : info[4]})
    

Citizen Francesca Bianchessi is paying 190000000000000000 wei --> 87.53485964055761 Euro
Citizen Ilaria Bolla is paying 490000000000000000 wei --> 225.74779591512228 Euro
Citizen Alessandro Botti is paying 1440000000000000000 wei --> 663.4220941179104 Euro
Citizen Davide Castellini is paying 120000000000000000 wei --> 55.285174509825865 Euro
Citizen Anna Di Marco is paying 216000000000000000 wei --> 99.51331411768656 Euro


In [15]:
get_citizen_info(dutiful_citizen)
"FINALLY PAID"


    Address eth : 0xeA31e61d90f3238DbF6C36E9aFcD555252383390
    Name : Francesca Bianchessi
    Family members : 2
    House sq.meters : 70
    Assigned weight\liters : 0
    TARI amount : 190000000000000000
    Recyclable Waste Tot : 0
    Non Recyclable Waste Tot : 0
    Paid TARI : True
    Active account : True
    


'FINALLY PAID'

<a name = 'cycle'></a>
## Working with trashbags lifecycles

The trucks will pick them up and drop them at the disposal station. From excel files we simulate what the sensors would use as inputs.

In [15]:
bags = pd.read_excel('data/example_data.xlsx', sheet_name='bags_data')
bags.head(6) 

,name,surname,weight,recycle
0,Francesca,Bianchessi,1,True
1,Ilaria,Bolla,2,False
2,Alessandro,Botti,3,True
3,Davide,Castellini,1,False
4,Anna,Di Marco,2,True
5,Francesca,Bianchessi,1,True


#### Pick up trash

Function called from the truck and using sensors to check the weight 

(Here we automatically assign the correct truck to the trash type. It would otherwise throw an error)

In [16]:
# The right truck will pick up the various trashbags
for i, name, sur, w, recyclable in bags.itertuples():

    # get address of generator
    generator_addr = data[data.name == name]['address'].item()
    
    # get correct truck address via subsetting
    truck_addr = data[(data['name'] == 'Truck') & (data['recycle'] == recyclable)]['address'].item()
    
    # 'pick' function in contract is called by the correct truck
    contract.functions.pick(generator_addr, w, random.randint(0, 1e+10)).transact({'from' : truck_addr})

In [17]:
get_citizen_info(dutiful_citizen)
get_truck_info(ex_truck)

"""See both waste counters increased for the citizen and weight transported for the truck"""


    Address eth : 0x20916136FcC20B2cB78FB476156FADB4b1641331
    Name : Francesca Bianchessi
    Family members : 2
    House sq.meters : 70
    Assigned weight\liters : 0
    TARI amount : 190000000000000000
    Recyclable Waste Tot : 16
    Non Recyclable Waste Tot : 12
    Paid TARI : True
    Active account : True
    

    Address eth : 0x87A34C6cB35E8a34B4Dc50f5746B533fB9076F83
    Truck number : 1
    Weight transported : 71
    Recyclable Truck : True
    Active Truck : True
    


'See both waste counters increased for the citizen and weight transported for the truck'

#### Drop bags at station

Still a function from the side of the truck. The truck will result empty after dropping and the station will increase its total counter with the recieved weight.

In [18]:
gps = pd.read_excel('data/example_data.xlsx', sheet_name='gps_data')
gps

,name,surname,lat,long
0,Disposal Station,Milano,45.4642,9.1900
1,Disposal Station,Bologna,44.4949,11.3426


In [19]:
get_station_info(ex_station)


    Address eth : 0xffE61522eF61ecFDF58020690C2CC340ccE667aC
    Station nr. : 1
    Weight : 0
    latitude : 45
    longitude : 9
    Recyclable Plant : True
    Active Plant : True
    


In [20]:
# get station name and coords from the gps
for i, name, sur, lat, long in gps.itertuples():
    
    #get station address
    station_addr = data[(data.name == name) & (data.surname == sur)]['address'].item()
    
    #get_station_info(station_addr)
    # need to get the type of station (recycling/not recycling) to pair the truck
    s_info = contract.functions.stations(station_addr).call() # s_info[4] is the type
    
    # pairing truck based on recyclable or not and calling the 'drop' function
    correct_truck = data[(data.name == 'Truck') & (data.recycle == s_info[4])]['address'].item()
    
    contract.functions.drop(station_addr, int(lat), int(long)).transact({'from' : correct_truck})
    

In [21]:
get_station_info(ex_station)
get_truck_info(ex_truck)

"""Station weight increased by weight in truck which is in turn empty"""


    Address eth : 0xffE61522eF61ecFDF58020690C2CC340ccE667aC
    Station nr. : 1
    Weight : 71
    latitude : 45
    longitude : 9
    Recyclable Plant : True
    Active Plant : True
    

    Address eth : 0x87A34C6cB35E8a34B4Dc50f5746B533fB9076F83
    Truck number : 1
    Weight transported : 0
    Recyclable Truck : True
    Active Truck : True
    


'Station weight increased by weight in truck which is in turn empty'

#### Recieved 

Function called by the station. Its sensors will say which truck deposited garbage and the total garbage present at the station in that moment. 

Right now we filled and emptied the truck in one go. So it's not much of an example (could rerun both for loops above -pick- -drop- and modify this excel to simulate multiple runs and the increased total weight of the station)

In [22]:
station_sense = pd.read_excel('data/example_data.xlsx', sheet_name='stations_data')
station_sense

,name,surname,weight
0,Truck,Recycle,71
1,Truck,Not Recycle,69


In [23]:
for i, name, sur ,weight in station_sense.itertuples():
    
    # get truck address
    truck_addr = data[(data.name == name) & (data.surname == sur)]['address'].item()
    
    # correct station to read the correct type of truck [otherwise we get an error]
    info_tr = contract.functions.trucks(truck_addr).call() # to get truck type in info_tr[2] 
    st_addr = data[(data.name == 'Disposal Station') & (data.recycle == info_tr[2])]['address'].item() 
    print(f"From Truck: {truck_addr}\nTo Station: {st_addr}")
    
    #recieved function call
    contract.functions.received(info_tr[2], truck_addr, weight).transact({'from' : st_addr})
    
    

From Truck: 0x87A34C6cB35E8a34B4Dc50f5746B533fB9076F83
To Station: 0xffE61522eF61ecFDF58020690C2CC340ccE667aC
From Truck: 0x13AF23dce57BBc6Edc73b2B0F71B0925d04A4F9F
To Station: 0x3E22eb9c92123B1D278D18C371Db54D7b655ed07


<a name = 'refund'></a>
## Refund

municipality withdraws form the contract the ETH and then does the payout for all citizens


In [24]:
print(municipality) # we have the municipality address stored from before

0xa14Dc72900593770d7aC81478660e91704da60CD


In [25]:
# the withdraw works just like this, see ganache '0' account --> municipality
before = web3.eth.getBalance(municipality)
contract.functions.withdraw().transact({'from' : municipality}) 
after = web3.eth.getBalance(municipality)
print(f"Withdrawn: {(after-before)/(10**18)} ETH (also minus gas costs)")

Withdrawn: 2.16024146 ETH (also minus gas costs)


***givePayout shoud check if _isAppropriateTime before giving refunds, we've commented that line to make it run***

In [26]:
# citizen address as input to function
for citizen in data[data.role == 'citizen']['address']:
    
    # balance comparison for the refund
    before = web3.eth.getBalance(citizen)
    contract.functions.givePayout(citizen).transact({'from' : municipality})
    after = web3.eth.getBalance(citizen)
    
    print(f"{citizen}\nRefunded wei :{after-before} --> Euro : {(after-before)*10**(-18)}\n")
        

0x20916136FcC20B2cB78FB476156FADB4b1641331
Refunded wei :9500000000000000 --> Euro : 0.009500000000000001

0xced646C5b0f38FbF0cca57a174A248907fd1132F
Refunded wei :9800000000000000 --> Euro : 0.009800000000000001

0x0Dc2998f5b8D55743B5b73756b8d526ce156aA19
Refunded wei :72000000000000000 --> Euro : 0.07200000000000001

0xA91d26a3863004aBd9613E955fF4842f2922A842
Refunded wei :2400000000000000 --> Euro : 0.0024000000000000002

0x132A9dFcBD4192aCE3a9dF87AdBA3FeE2191B7a7
Refunded wei :10800000000000000 --> Euro : 0.0108



In [27]:
# how much is still on the contract should be low
balance = contract.functions.MunicipalityBalance().call({'from' : municipality})
print('On contract there is still :', balance/(10**18), 'ETH')

On contract there is still : 0.19022 ETH


In [28]:
# selfdestruct contract by invoking kill function
contract.functions.destroyContract().transact({'from' : municipality})

HexBytes('0xcfb71e13c7eccc2cc1e4e553e112e82bbc449ffd14e1b3d19548d69f703b8b6c')

<a name = 'logs'></a>

## Event/Log Filtering

After running the above cells the chain is populated with a lot of different logs from events. They can be extracted with specific filters that are contract specific. 

(Right now just 1 filter for the sake of demonstration and an example of the database that is created. A ***real time*** version that repeatedly checks the last block is in the 'events_real_time.py')

In [29]:
# creating logs/event filters to inspect the whole chain (fromBlock = 1)

# filter for the Event 'PickedUp' generated by the truck
pickedUp_filter = contract.events.PickedUp.createFilter(fromBlock = 1)
#recieved_filter etc

filters = [pickedUp_filter]

In [30]:
def get_past_logs(filter_list):
    """
    Iterates over every filter built and extracts logs from the block specified in the filter to the 'latest'
    
    inputs
    ---------------
     filter_list : filters created
    
    returns
    ---------------
     list containing every event attribute generated from the 'emit' on the contract
    """
    events = []
    for event_filter in filter_list:
        for e in event_filter.get_all_entries(): # get_new_entry() to check only last block
            
            # e is a nested dictionary, like this we bring all elements on the same level
            args = dict(e['args'])
            args.update(dict(e))
            del args['args'] # brought one level above, delete the nest
            # args.pop('args', None) # could delete like this too
            
            events.append(args)
            
    return events

In [31]:
# Pass filters on the chain (from block 1) and pick logs found (for now only 'ToPickUp')
# Can see logs of the 5 citizens, each one a different generator and all the garble generated by the event
events = get_past_logs(filters)
df = pd.DataFrame(events)
df.head()

,transporter,wasteType,bagId,generator,wasteWeight,pickUpTime,event,logIndex,transactionIndex,transactionHash,address,blockHash,blockNumber
0,0x87A34C6cB35E8a34B4Dc50f5746B533fB9076F83,True,b'l\x87C\xe9\xca\xe6\xbc\xe2\xd3|\x9f3\xbf\xd1...,0x20916136FcC20B2cB78FB476156FADB4b1641331,1,1607765243,PickedUp,0,0,"b':\xe12\x02\x00\xce\x1f\x98\xf0\xf1Z\xca,\n\x...",0x2F14C8d0eA57c4b2058f82106C0dbBFcDaD3A9Bc,b'\x98\xb6\xc9|\x07\xb7\xee\x8b\r\xf4\xba\xd3\...,21
1,0x13AF23dce57BBc6Edc73b2B0F71B0925d04A4F9F,False,b'k8\x8d\x10B\xe5\x13\x87\xcbu;\xe2\xf6\xab\x0...,0xced646C5b0f38FbF0cca57a174A248907fd1132F,2,1607765244,PickedUp,0,0,b'\x03HHY\x7f\xebc\xc7\xcb\xd3gD\x9e\xe1\xd3M\...,0x2F14C8d0eA57c4b2058f82106C0dbBFcDaD3A9Bc,b'u\x1b\x1aK-\x82\x7f\xe0\xcd\xe3\xfbW\xf3 SX\...,22
2,0x87A34C6cB35E8a34B4Dc50f5746B533fB9076F83,True,b'EZ\x95\xf3\x92\xeeC\xd4c\\?C\xb8\xc6\xa6We\x...,0x0Dc2998f5b8D55743B5b73756b8d526ce156aA19,3,1607765245,PickedUp,0,0,b'\x85\xf9B\xb4\x07\x7f\xec\xb1\xe1\x8d\xc1d\x...,0x2F14C8d0eA57c4b2058f82106C0dbBFcDaD3A9Bc,b'Q\x80\x0c\xf4\xfcK\x8c?\x89\xf0M\x06c\x89\xb...,23
3,0x13AF23dce57BBc6Edc73b2B0F71B0925d04A4F9F,False,b'\x11\xfd\xe5\xfb\xa4\xa1\xf9j\x85\xd3K\xe7[\...,0xA91d26a3863004aBd9613E955fF4842f2922A842,1,1607765246,PickedUp,0,0,b'\xbc\x05a\xbe\x13\xc4\xc9\x95e\xac3\xac\x9f\...,0x2F14C8d0eA57c4b2058f82106C0dbBFcDaD3A9Bc,"b'\x8f\x87n?VOh\xaf,\x011\x06V\xda\xdf\xb2\x7f...",24
4,0x87A34C6cB35E8a34B4Dc50f5746B533fB9076F83,True,b'\xac\x1c\xf9kx\x06\xab5\xb4\xb3\xd4\xf0{\xc1...,0x132A9dFcBD4192aCE3a9dF87AdBA3FeE2191B7a7,2,1607765246,PickedUp,0,0,"b'p\xf6<o\xe5p~\x87J\xb8,\x0b\\\xe1\x89\t\xa7\...",0x2F14C8d0eA57c4b2058f82106C0dbBFcDaD3A9Bc,b'e\xb7Yg&\xed\x92G\xa4\xf5\xe3Ji\\\x9bk\x94\x...,25
